<a href="https://colab.research.google.com/github/hideaki-kyutech/syseng2025/blob/main/ex_pyomo_minlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MINLPの実行例
- Objective Function:
  - $max(x+y)$
- Constraints:
  - $-x+2xy\leq 7$
  - $2x+y\leq 14$
  - $2x-y\leq 10$
  - $0\leq x\leq 10$
  - $0\leq y\leq 10$
  - $x$ as integer

## pyomoフレームワークのインストール
- Colabにはすでにインストール済みのため不要

In [2]:
!pip install -q pyomo

In [14]:
!apt-get install -y -qq glpk-utils

In [17]:
!pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 35.4 MB/s eta 0:00:00


In [3]:
!wget -N -q "https://github.com/thomasfork/ipopt_linux/raw/main/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

## pyomoのインポート

In [4]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [44]:
from amplpy import modules
!python -m amplpy.modules install coin highs scip gcg -q  # Install HiGHS, CBC, Couenne, Bonmin, Ipopt, SCIP, and GCG

## solverの設定

In [45]:
model = pyo.ConcreteModel()

## 変数の設定

In [46]:
model.x = pyo.Var(bounds=(0,10), within=Integers)
model.y = pyo.Var(bounds=(0,10))
x = model.x
y = model.y

## 制約式の設定

In [47]:
model.C1 = pyo.Constraint(expr= -x+2*x*y<=7)
model.C2 = pyo.Constraint(expr= 2*x+y<=14)
model.C3 = pyo.Constraint(expr= 2*x-y<=10)

## 目的関数の設定

In [48]:
model.obj = pyo.Objective(expr= x+y, sense=maximize)

## 解を求める

In [49]:
solver_name = "couenne"  # "highs", "cbc",  "couenne", "bonmin", "ipopt", "scip", or "gcg".

In [50]:
opt = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
opt.solve(model)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 2, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'couenne\\x3a Optimal', 'Termination condition': 'optimal', 'Id': 3, 'Error rc': 0, 'Time': 0.04151773452758789}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## 問題設定のサマリー

In [51]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :    10 : False : False : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x + y

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :  -Inf : - x + 2*x*y :   7.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :  14.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x - y :  10.0 :   True

6 Declarations: x y C1 C2 C3 obj


## 結果の表示

In [52]:
x_value = pyo.value(x)
y_value = pyo.value(y)
print('x=',x_value)
print('y=',y_value)

x= 0.0
y= 10.0
